In [1]:
import tensorflow as tf
import spektral 
import numpy as np 

In [2]:
# If you encounter InternalError: Blas xGEMM launch failed  error 
# I did encounter !! :((
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [3]:
cora= spektral.datasets.citation.Citation('cora',random_split=True,dtype= np.float32)

C:\Users\MSI\anaconda3\envs\tensorflow_learning\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [4]:
cora_g =  cora.read()[0]
type(cora_g)

C:\Users\MSI\anaconda3\envs\tensorflow_learning\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


spektral.data.graph.Graph

In [5]:
# `x`: np.array, the node features (shape `(n_nodes, n_node_features)`);
# `a`: np.array or scipy.sparse matrix, the adjacency matrix (shape `(n_nodes, n_nodes)`);
# `e`: np.array, the edge features (shape `(n_nodes, n_nodes, n_edge_features)` or `(n_edges, n_edge_features)`);
# `y`: np.array, the node or graph labels (shape `(n_nodes, n_labels)` or `(n_labels, )`);

In [6]:
adj= cora_g.a # note that this is a numpy sparse matrix 
features= cora_g.x
labels=cora_g.y
train_mask=cora.mask_tr
val_mask=cora.mask_va
test_mask= cora.mask_te

In [7]:
adj = adj.todense()+np.eye(adj.shape[0])
adj = adj.astype('float32')

In [8]:
print('type of adj: ',adj.dtype,' type of features: ', features.dtype)

type of adj:  float32  type of features:  float32


In [9]:
print(f"train size: {np.sum(train_mask)}")
print(f"validation size: {np.sum(val_mask)}")
print(f"test size: {np.sum(test_mask)}")
print(f"number of node features: {cora.n_node_features}")

train size: 140
validation size: 210
test size: 2358
number of node features: 1433


In [10]:
def masked_softmax_cross_entropy(logits,labels,mask): #loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=labels)
    mask = tf.cast(mask , dtype = tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)

def masked_accuracy(logits,labels, mask):
    correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(labels,1))
    accuracy_all = tf.cast(correct_prediction,tf.float32)
    mask = tf.cast(mask,tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)

In [11]:
#defining GCN layer without normalization 
def gcn_layer(fts, adj, transform, activation):  
    seq_fts = transform(fts)
    ret_fts = tf.matmul(adj,seq_fts)
    return activation(ret_fts) 

# creating custom layer 
# class GNNlayer(tf.keras.Model):
    

In [12]:
from tqdm.auto import tqdm,trange
from time import sleep 
def train_cora(fts,adj,gnn_lyr,units, epochs ,lr ):
    
    best_accuracy=0.0
    
    layer1 = tf.keras.layers.Dense(units) # transform function for first GNN layer
    layer2 = tf.keras.layers.Dense(7) # transform function for second GNN layer 
    
    def cora_gnn(fts,adj):   # entire GNN architecutre
        hidden = gnn_lyr(fts,adj,layer1,tf.nn.relu) # first layer of the GNN architecture 
        logits = gnn_lyr(hidden,adj,layer2, tf.identity) 
        return logits 
    #custom training loop
    optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
    
    
    def getMetrics(predicts):
        
        nonlocal best_accuracy
        val_accuracy = masked_accuracy(predicts, labels, val_mask)
        test_accuracy = masked_accuracy(predicts, labels, test_mask)
        
        if val_accuracy>best_accuracy:
            best_accuracy=val_accuracy
            tqdm.write("\rEpoch {}/{} | Training loss {:.4f} |  Val accuracy {:.4f} | Test accuracy {:.4f}".
                  format(epoch,epochs,loss.numpy(),val_accuracy.numpy(),test_accuracy.numpy()),
                 end="\n" if epoch<epochs+1 else "" )
            
    for epoch in trange(1,epochs+1):
        with tf.GradientTape() as tape:
            logits = cora_gnn(fts,adj)  #calculated logits 
            loss = masked_softmax_cross_entropy(logits, labels, train_mask)
            
        variables = tape.watched_variables()
        if epoch==1 :
            print(len(variables))
        gradients = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(gradients, variables))
        
        # I'm gonna calculate predicts again 
        predicts= cora_gnn(fts,adj)
        sleep(0.001)
        getMetrics(predicts)
    

In [13]:
train_cora(features, adj, gcn_layer, 32, 200, 0.01)

  0%|          | 0/200 [00:00<?, ?it/s]

4
Epoch 1/200 | Training loss 3.8084 |  Val accuracy 0.3524 | Test accuracy 0.3507
Epoch 3/200 | Training loss 3.9402 |  Val accuracy 0.4905 | Test accuracy 0.4695
Epoch 4/200 | Training loss 3.2394 |  Val accuracy 0.5095 | Test accuracy 0.4873
Epoch 5/200 | Training loss 2.7456 |  Val accuracy 0.5952 | Test accuracy 0.5653
Epoch 6/200 | Training loss 1.9726 |  Val accuracy 0.6810 | Test accuracy 0.6459
Epoch 7/200 | Training loss 1.3326 |  Val accuracy 0.7952 | Test accuracy 0.7222


In [14]:
#degree matrix of adjacency matrix adj
deg = tf.reduce_sum(adj, axis = -1)
# print("adjacency matrix",adj,"\n")
# print("degree matrix",deg,"\n")
# print(adj/deg)

In [15]:
# train gcn using mean pooling 
train_cora(features, adj/deg,gcn_layer, 32, 200, 0.01)

  0%|          | 0/200 [00:00<?, ?it/s]

4
Epoch 1/200 | Training loss 1.9495 |  Val accuracy 0.3429 | Test accuracy 0.3308
Epoch 2/200 | Training loss 1.7526 |  Val accuracy 0.3667 | Test accuracy 0.3520
Epoch 3/200 | Training loss 1.5504 |  Val accuracy 0.4000 | Test accuracy 0.3919
Epoch 4/200 | Training loss 1.3480 |  Val accuracy 0.4571 | Test accuracy 0.4741
Epoch 5/200 | Training loss 1.1526 |  Val accuracy 0.5619 | Test accuracy 0.5615
Epoch 6/200 | Training loss 0.9717 |  Val accuracy 0.6571 | Test accuracy 0.6383
Epoch 7/200 | Training loss 0.8127 |  Val accuracy 0.6810 | Test accuracy 0.6845
Epoch 8/200 | Training loss 0.6741 |  Val accuracy 0.7143 | Test accuracy 0.7125
Epoch 9/200 | Training loss 0.5522 |  Val accuracy 0.7714 | Test accuracy 0.7328
Epoch 10/200 | Training loss 0.4466 |  Val accuracy 0.8000 | Test accuracy 0.7506
Epoch 11/200 | Training loss 0.3581 |  Val accuracy 0.8095 | Test accuracy 0.7621
Epoch 12/200 | Training loss 0.2850 |  Val accuracy 0.8143 | Test accuracy 0.7714
Epoch 17/200 | Training

In [16]:
norm_deg = tf.linalg.diag(1.0/tf.sqrt(deg))
norm_adj= tf.matmul(norm_deg,tf.matmul(adj,norm_deg))

In [17]:
train_cora(features, norm_adj, gcn_layer, 32, 200, 0.01)

  0%|          | 0/200 [00:00<?, ?it/s]

4
Epoch 1/200 | Training loss 1.9416 |  Val accuracy 0.4143 | Test accuracy 0.4402
Epoch 2/200 | Training loss 1.7342 |  Val accuracy 0.4476 | Test accuracy 0.4754
Epoch 3/200 | Training loss 1.5157 |  Val accuracy 0.5143 | Test accuracy 0.5352
Epoch 4/200 | Training loss 1.2833 |  Val accuracy 0.5714 | Test accuracy 0.5895
Epoch 5/200 | Training loss 1.0641 |  Val accuracy 0.6190 | Test accuracy 0.6429
Epoch 6/200 | Training loss 0.8676 |  Val accuracy 0.6857 | Test accuracy 0.6764
Epoch 7/200 | Training loss 0.6967 |  Val accuracy 0.7095 | Test accuracy 0.7044
Epoch 9/200 | Training loss 0.4342 |  Val accuracy 0.7238 | Test accuracy 0.7375
Epoch 10/200 | Training loss 0.3397 |  Val accuracy 0.7476 | Test accuracy 0.7464
Epoch 11/200 | Training loss 0.2649 |  Val accuracy 0.7524 | Test accuracy 0.7557
Epoch 12/200 | Training loss 0.2058 |  Val accuracy 0.7714 | Test accuracy 0.7629
Epoch 13/200 | Training loss 0.1592 |  Val accuracy 0.7810 | Test accuracy 0.7680
Epoch 15/200 | Trainin

In [18]:
# for a moment let's make the adj marix orginal matrix
adj_org = cora_g.a.todense()
adj_org = adj_org.astype('float32')

In [21]:
from tqdm.auto import tqdm,trange
from time import sleep 
def train_cora_diff(fts,adj,gnn_lyr,units, epochs ,lr ):
    
    best_accuracy=0.0
    
    layer1 = tf.keras.layers.Dense(units) # transform function for first GNN layer
    layer2 = tf.keras.layers.Dense(7) # transform function for second GNN layer 
    
    def cora_gnn(fts,adj):   # entire GNN architecutre
        l = tf.Variable(initial_value=1, trainable = True , dtype = tf.float32)
        adj = adj + l*np.eye(adj.shape[0])
        hidden = gnn_lyr(fts,adj,layer1,tf.nn.relu) # first layer of the GNN architecture 
        logits = gnn_lyr(hidden,adj,layer2, tf.identity) 
        return logits 
    #custom training loop
    optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
     
    def getMetrics(predicts):
        
        nonlocal best_accuracy
        val_accuracy = masked_accuracy(predicts, labels, val_mask)
        test_accuracy = masked_accuracy(predicts, labels, test_mask)
        
        if val_accuracy>best_accuracy:
            best_accuracy=val_accuracy
            tqdm.write("\rEpoch {}/{} | Training loss {:.4f} |  Val accuracy {:.4f} | Test accuracy {:.4f}".
                  format(epoch,epochs,loss.numpy(),val_accuracy.numpy(),test_accuracy.numpy()),
                 end="\n" if epoch<epochs+1 else "" )

    for epoch in trange(1,epochs+1):
        with tf.GradientTape() as tape:
           
            logits = cora_gnn(fts,adj)  #calculated logits 
            loss = masked_softmax_cross_entropy(logits, labels, train_mask)
            
        variables = tape.watched_variables()
        if epoch==1 :
            print("num of watching variables: ", len(variables))
        gradients = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(gradients, variables))
        
        # I'm gonna calculate predicts again 
        predicts= cora_gnn(fts,adj)
        #predicts = logits
        sleep(0.001)
        getMetrics(predicts)
        
        if epoch==epochs:
            print("trade-off constant: ", variables[-1].numpy())
    

In [22]:
train_cora_diff(features, adj_org, gcn_layer, 32, 200, 0.01)

  0%|          | 0/200 [00:00<?, ?it/s]

num of watching variables:  5
Epoch 1/200 | Training loss 3.9408 |  Val accuracy 0.2952 | Test accuracy 0.3240
Epoch 2/200 | Training loss 6.3967 |  Val accuracy 0.3429 | Test accuracy 0.3707
Epoch 3/200 | Training loss 2.5959 |  Val accuracy 0.4952 | Test accuracy 0.5106
Epoch 4/200 | Training loss 1.5101 |  Val accuracy 0.5571 | Test accuracy 0.5237
Epoch 5/200 | Training loss 1.2208 |  Val accuracy 0.6762 | Test accuracy 0.6387
Epoch 6/200 | Training loss 0.9422 |  Val accuracy 0.6857 | Test accuracy 0.6832
Epoch 8/200 | Training loss 0.6520 |  Val accuracy 0.6952 | Test accuracy 0.7095
Epoch 9/200 | Training loss 0.4889 |  Val accuracy 0.7286 | Test accuracy 0.7129
Epoch 10/200 | Training loss 0.3200 |  Val accuracy 0.7381 | Test accuracy 0.7218
Epoch 11/200 | Training loss 0.2555 |  Val accuracy 0.7619 | Test accuracy 0.7193
Epoch 22/200 | Training loss 0.0801 |  Val accuracy 0.7667 | Test accuracy 0.7252
Epoch 25/200 | Training loss 0.0588 |  Val accuracy 0.7810 | Test accuracy 0